# Export variables for Taylor Diagrams
- This script is used to export variables from simulations and Taylor Diagram metrics;
- Simulations: TRAF and sensitity simulations at FR-Capitole and UK-Manchester. 

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
period_list = ['summer', 'winter'] 
factor_list = ['add0.1', 'add0.2', 'add0.4', 'add0.8', 'sub0.1', 'sub0.2', 'sub0.4', 'sub0.8']

## FR-Capitole

In [3]:
start = datetime.fromisoformat('2004-02-20T00:30:00')
summer_end = datetime.fromisoformat('2004-06-27T00:00:00')  # just an example
winter_end = datetime.fromisoformat('2005-01-02T00:00:00')  # just an example

interval = timedelta(minutes=30)
summer_n_timesteps = int((summer_end - start) / interval)
winter_n_timesteps = int((winter_end - start) / interval)
print(summer_n_timesteps, winter_n_timesteps)

6143 15215


In [9]:
date_list = ['2004-06-27-01800', '2005-01-02-01800']
start_date_list = ['2004-06-27T00:00:00', '2005-01-02T00:00:00']
end_date_list = ['2004-07-04T00:00:00', '2005-01-09T00:00:00']
season_list = ['summer', 'winter']
var1 = ['LWup', 'Qh', 'Qle', 'Qtau']
var2 = ['FIRE_U', 'FSH_U', 'EFLX_LH_TOT_U', 'TAUX']
rename_dict = dict(zip(var2, var1))
GRIDNAME='FR-Capitole'
gridname='FR-Cap'
observation = f'{home_path}0_lcz_sp/UrbanPlumber/Urban-PLUMBER_FullCollection_v1/' + GRIDNAME + '/timeseries/'+ GRIDNAME + '_clean_observations_v1.nc'
ds_ob = xr.open_dataset(observation)
ds_traffic = xr.open_dataset(f'{home_path}0_urban_traffic/archive/{gridname}_traffic/lnd/hist/{gridname}_traffic.clm2.h0.2004-02-20-03600.nc')
for p, period in enumerate(period_list):
    ds_ob_sel = ds_ob.sel(time=slice(start_date_list[p], end_date_list[p]))
    df_obs = ds_ob_sel[var1].to_dataframe().reset_index()
    df_obs['case'] = 'obs'
    #df_obs = df_obs.iloc[:-1]
    df_obs=df_obs.iloc[1:]
    season = season_list[p]
    casename = f'{gridname}_{season}'
    date = date_list[p]
    all_dfs = [df_obs]
    ds_traffic_sel = ds_traffic.sel(time=slice(start_date_list[p], end_date_list[p]))
    df_traffic = ds_traffic_sel[var2].to_dataframe().reset_index()
    df_traffic['time'] = pd.to_datetime(df_traffic['time'])
    df_traffic['time'] = df_traffic['time'].dt.round('min').dt.ceil('min')
    df_traffic.drop(columns=['lndgrid'], inplace=True)
    df_traffic = df_traffic.rename(columns=rename_dict) 
    df_traffic['case'] = 'traffic'
    df_traffic['Qtau'] = -df_traffic['Qtau']
    df_traffic=df_traffic.iloc[1:]
    for factor in factor_list:
        ds_factor = xr.open_dataset(f'{home_path}0_urban_traffic/archive/sensitivity/{casename}/{casename}.clm2.h0.{date}_{factor}.nc')
        df_factor = ds_factor[var2].to_dataframe().reset_index()
        df_factor['time'] = pd.to_datetime(df_factor['time'])
        df_factor['time'] = df_factor['time'].dt.round('min').dt.ceil('min')
        df_factor.drop(columns=['lndgrid'], inplace=True)
        df_factor = df_factor.rename(columns=rename_dict) 
        df_factor['case'] = factor
        df_factor['Qtau'] = -df_factor['Qtau']
        all_dfs.append(df_factor)
    all_dfs.append(df_traffic)    
    df_combined = pd.concat(all_dfs, ignore_index=True)        
   # df_combined.to_csv(f'./output/{GRIDNAME}_{season}.csv', index=False)
    df_combined.head()               

## UK-Manchester

In [164]:
start = datetime.fromisoformat('2022-01-01T00:00:00')
summer_end = datetime.fromisoformat('2022-07-16T00:00:00')  # just an example
winter_end = datetime.fromisoformat('2022-12-10T00:00:00')  # just an example

interval = timedelta(minutes=60)
summer_n_timesteps = int((summer_end - start) / interval)
winter_n_timesteps = int((winter_end - start) / interval)
print(summer_n_timesteps, winter_n_timesteps)

4704 8232


In [165]:
sensor_id = 'MOD-PM-00427'
df_sensor = pd.read_csv(f'{home_path}0_lcz_mcr/output_analysis/single_point/calibration/adjusted_sensor_data/{sensor_id}.csv')
df_sensor.rename(columns={'timestamp': 'time'}, inplace=True)
df_sensor.head()

,time,rh_hourly_avg,temp_hourly_avg,hour,month,std_temp_bias,std_rh_bias,rh_hourly_avg_corrected,temp_hourly_avg_temp,temp_hourly_avg_corrected
0,2022-02-21 14:00:00,43.610000,17.06000,14,2,0.196525,-0.182844,51.583847,13.707289,13.707289
1,2022-02-21 15:00:00,39.354386,18.73386,15,2,0.191343,-0.178718,46.387718,15.149258,15.149258
2,2022-02-21 16:00:00,NaN,NaN,16,2,0.176080,-0.158543,NaN,NaN,NaN
3,2022-02-21 17:00:00,NaN,NaN,17,2,0.156802,-0.136305,NaN,NaN,NaN
4,2022-02-21 18:00:00,NaN,NaN,18,2,0.148477,-0.124433,NaN,NaN,NaN


In [167]:
date_list = ['2022-07-16-03600', '2022-12-10-03600']
start_date_list = ['2022-07-15T00:00:00', '2022-12-09T00:00:00']
start_date_list2 = ['2022-07-16T01:00:00', '2022-12-10T01:00:00']
end_date_list = ['2022-07-23-T00:00:00', '2022-12-17T00:00:00']
var3 = ['temp_hourly_avg_corrected', 'rh_hourly_avg_corrected']
var4 = ['TSA_U', 'RH2M']
rename_dict = dict(zip(var3, var4))
GRIDNAME='UK-MCR'

ds_traffic = xr.open_dataset(f'{home_path}0_urban_traffic/archive/{GRIDNAME}_traffic/lnd/hist/{GRIDNAME}_traffic.clm2.h0.2022-01-01-03600.nc')
for p, period in enumerate(period_list):
    df_obs = df_sensor[(df_sensor['time']>=start_date_list[p]) & (df_sensor['time']<=end_date_list[p])].reset_index(drop=True)
    df_obs_sel = df_obs[['time'] + var3][1:-23].reset_index(drop=True)
    df_obs_sel['time']= pd.to_datetime(df_obs_sel['time'])
    df_obs_sel.rename(columns=rename_dict, inplace=True)
    df_obs_sel['case'] = 'obs'
    season = season_list[p]
    casename = f'{GRIDNAME}_{season}'
    date = date_list[p]
    all_dfs = [df_obs_sel]
    ds_traffic_sel = ds_traffic.sel(time=slice(start_date_list2[p], end_date_list[p]))
    df_traffic = ds_traffic_sel[var2].to_dataframe().reset_index()
    df_traffic.drop(columns=['lndgrid'], inplace=True)
    df_traffic['time'] = df_traffic['time'].dt.round('min').dt.ceil('min')
    df_traffic['case'] = 'traffic'
    df_traffic['TSA_U'] = df_traffic['TSA_U'] -273.15
    for factor in factor_list:
        ds_factor = xr.open_dataset(f'{home_path}0_urban_traffic/archive/sensitivity/{casename}/{casename}.clm2.h0.{date}_{factor}.nc')
        df_factor = ds_factor[var4].to_dataframe().reset_index()
        df_factor['time'] = pd.to_datetime(df_factor['time'])
        df_factor['time'] = df_factor['time'].dt.round('min').dt.ceil('min')
        df_factor.drop(columns=['lndgrid'], inplace=True)
        df_factor['case'] = factor
        df_factor['TSA_U'] = df_factor['TSA_U'] -273.15
        all_dfs.append(df_factor)
    all_dfs.append(df_traffic)    
    df_combined = pd.concat(all_dfs, ignore_index=True)        
    df_combined.to_csv(f'./output/{GRIDNAME}_{season}.csv', index=False)
    df_combined.head()      

## Taylor diagram metrics

In [138]:
case_list = factor_list + ['traffic']

In [5]:
GRIDNAME='FR-Capitole'
for season in season_list:
    df_combined = pd.read_csv(f'./output/{GRIDNAME}_{season}.csv')
    df_obs = df_combined[df_combined['case'] == 'obs'].reset_index(drop=True)
    std_result = []
    coef_result = []
    for factor in case_list:
        df_factor = df_combined[df_combined['case'] == factor].reset_index(drop=True)
        for var in var1:
            df_sim = df_factor[['time', var]].copy()
            df_sim = df_sim.rename(columns={var: 'sim'})
            df_sim['obs'] = df_obs[var]
            df_sim.loc[df_sim['obs'].isna(), ['sim']] = np.nan
            df_sim.dropna(subset=['obs', 'sim'], inplace=True)
            std = float(np.std(df_sim['sim']))
            sdev = std/float(np.std(df_sim['obs']))
            coef = float(xr.corr(xr.DataArray(df_sim['obs']), xr.DataArray(df_sim['sim'])).values)
            std_result.append(sdev)
            coef_result.append(coef)
    df = pd.DataFrame({'factor': np.repeat(case_list, len(var1)), 'var': var1 * len(case_list), 'sdev': std_result, 'coef': coef_result})  
    df.to_csv(f'./data_for_figure/{GRIDNAME}_{season}_std_coef.csv', index=False)      
    df 

In [172]:
GRIDNAME='UK-MCR'
for season in season_list:
    df_combined = pd.read_csv(f'./output/{GRIDNAME}_{season}.csv')
    df_obs = df_combined[df_combined['case'] == 'obs'].reset_index(drop=True)
    std_result = []
    coef_result = []
    for factor in case_list:
        df_factor = df_combined[df_combined['case'] == factor].reset_index(drop=True)
        for var in var4:
            df_sim = df_factor[['time', var]].copy()
            df_sim = df_sim.rename(columns={var: 'sim'})
            df_sim['obs'] = df_obs[var]
            df_sim.loc[df_sim['obs'].isna(), ['sim']] = np.nan
            df_sim.dropna(subset=['obs', 'sim'], inplace=True)
            std = float(np.std(df_sim['sim']))
            sdev = std/float(np.std(df_sim['obs']))
            coef = float(xr.corr(xr.DataArray(df_sim['obs']), xr.DataArray(df_sim['sim'])).values)
            std_result.append(sdev)
            coef_result.append(coef)
    df = pd.DataFrame({'factor': np.repeat(case_list, len(var4)), 'var': var4 * len(case_list), 'sdev': std_result, 'coef': coef_result})  
    df.to_csv(f'./data_for_figure/{GRIDNAME}_{season}_std_coef.csv', index=False)      
    df 